In [1]:
import pandas as pd
from scipy.stats import ttest_ind
class color:
    BOLD = '\033[1m'
    END = '\033[0m'
    GREEN = '\033[92m'

In [2]:
# Сделаем функцию, которая напишет выводы по итогам t-теста
def ttest_interpreter(s, p):
    if p < 0.05:
        print(color.GREEN + 'Выборки не равны' + color.END)
    elif p > 0.05:
        print('Выборки равны')
    if s < -1.96:
        print(color.BOLD + 'Тестовая группа совершила больше покупок' + color.END)
    elif s > 1.96:
        print(color.BOLD + 'Контрольная группа совершила больше покупок' + color.END)

In [3]:
df = pd.read_csv('Домашняя работа Бизнес-3 (исходник).csv', sep=';')
df['time_came'] = pd.to_datetime(df['time_came'], format='%d.%m.%Y') # Приведём дату к стандартному виду
df # Посмотрим на нашу табличку

,id_client,id_group,city,nflag_purchase,time_came
0,24360208,0,Moscow,1,2021-10-21
1,24360925,1,Moscow,1,2021-10-16
2,24357607,1,Saint-Petersbourg,0,2021-01-09
3,24362041,0,Saint-Petersbourg,0,2020-01-25
4,24360773,1,Novosibirsk,0,2021-04-03
...,...,...,...,...,...
6389,24356990,1,Moscow,1,2021-10-08
6390,24359007,0,Novosibirsk,1,2021-05-24
6391,24362669,1,Moscow,0,2021-05-20
6392,24357547,0,Novosibirsk,0,2021-01-13


***id_client*** - уникальный идентификатор клиента

***id_group*** - идентификатор группы (0 - контроль, 1 - тест)

***city*** - название города

***nflag_purchase*** - факт покупки (1 - приобретен рекламируемый товар, 0 - нет)

***time_came*** - дата регистрации пользователя в приложении

In [4]:
# Посмотрим на все данные, сделаем выводы

df_conv = df.groupby('id_group').agg({'nflag_purchase':'sum','id_client':'count'}).reset_index() # Аггрегируем группы
df_conv['conv(%)'] = round((df_conv['nflag_purchase'] / df_conv['id_client']), 4)*100 # Посчитаем конверсию
s, p = ttest_ind(df[df['id_group']==0]['nflag_purchase'], df[df['id_group']==1]['nflag_purchase'])

print('t-test control vs test all cities')
print('S:', s)
print('P:', p)
print('Среднее по контролю:', round(df[df['id_group']==0]['nflag_purchase'].mean(), 2))
print('Среднее по тесту:', round(df[df['id_group']==1]['nflag_purchase'].mean(), 2))

ttest_interpreter(s, p)
df_conv

t-test control vs test all cities
S: -2.4013624185110922
P: 0.01636252986305824
Среднее по контролю: 0.27
Среднее по тесту: 0.3
Выборки не равны
Тестовая группа совершила больше покупок


,id_group,nflag_purchase,id_client,conv(%)
0,0,872,3171,27.50
1,1,974,3223,30.22


Мы обнаружили, что тестовая группа совершила больше покупок рекламируемого товара, 
что подтверждают результаты t-теста. Однако стоит рассмотреть более детально сегменты по городам

In [5]:
# Посмотрим конверсии для контрольной и тестовой группы в каждом городе
df_city = df.groupby(['id_group','city']).agg({'nflag_purchase':'sum','id_client':'count'}).reset_index()
df_city['conv'] = df_city['nflag_purchase'] / df_city['id_client']
df_city.sort_values(by = 'city')

df_city

,id_group,city,nflag_purchase,id_client,conv
0,0,Kazan,142,439,0.323462
1,0,Moscow,304,1106,0.274864
2,0,Novosibirsk,152,585,0.259829
3,0,Saint-Petersbourg,206,807,0.255266
4,0,Vladivostok,68,234,0.290598
5,1,Kazan,171,503,0.339960
6,1,Moscow,311,1068,0.291199
7,1,Novosibirsk,166,609,0.272578
8,1,Saint-Petersbourg,238,815,0.292025
9,1,Vladivostok,88,228,0.385965


In [6]:
# Посчитаем t-тест по городам
for i in df['city'].unique():
    print('=======================t-test in', color.BOLD + i + color.END)
    s, p = ttest_ind(df[(df['city']==i)&(df["id_group"]==0)]['nflag_purchase'], 
                     df[(df['city']==i)&(df["id_group"]==1)]['nflag_purchase'])
    
    print('S:', s)
    print('P:', p)
    ttest_interpreter(s, p)

=======================t-test in Moscow
S: -0.8450837813559311
P: 0.3981571415772427
Выборки равны
=======================t-test in Saint-Petersbourg
S: -1.6604718327634895
P: 0.09701308927385824
Выборки равны
=======================t-test in Novosibirsk
S: -0.4978267258755046
P: 0.6186980029170557
Выборки равны
=======================t-test in Kazan
S: -0.5357666217562046
P: 0.592246594180279
Выборки равны
=======================t-test in Vladivostok
S: -2.1734327152064643
P: 0.030257214768841512
Выборки не равны
Тестовая группа совершила больше покупок


Нам удалось обнаружить, что только во Владивостоке СМС-уведомления привели к большим покупкам, чем при Push-уведомлениях.
Продолжим сегментацию. Рассмотрим клиентов с регистрацией 2020 и 2021 года в каждом городе

In [7]:
for i in df['city'].unique(): # Цикл в рамках каждого города

    print('t-test in', color.BOLD + i + color.END)

    for year in df['time_came'].dt.year.unique(): # И цикл в рамках каждого года регистрации
        print()
        print('>Registred in', year, 'year<')

        s, p = ttest_ind(df[(df['city'] == i) & (df['id_group'] == 1 & (df['time_came'].dt.year == year))]['nflag_purchase'], 
                         df[(df['city'] == i) & (df['id_group'] == 0) & (df['time_came'].dt.year == year)]['nflag_purchase'])

        print('S:', s)
        print('P:', p)
        ttest_interpreter(s, p)
        print()

    print('================================')

t-test in Moscow

>Registred in 2021 year<
S: 0.7891696050404378
P: 0.43011400847493275
Выборки равны


>Registred in 2020 year<
S: -0.38260125513496546
P: 0.7020724568609569
Выборки равны

t-test in Saint-Petersbourg

>Registred in 2021 year<
S: 1.0081040187611883
P: 0.313585715841074
Выборки равны


>Registred in 2020 year<
S: 0.32749312190772745
P: 0.7433584720132069
Выборки равны

t-test in Novosibirsk

>Registred in 2021 year<
S: 0.8752185166062159
P: 0.3816714573647946
Выборки равны


>Registred in 2020 year<
S: -0.6034615681477077
P: 0.5463718791584599
Выборки равны

t-test in Kazan

>Registred in 2021 year<
S: 0.263475349853756
P: 0.7922662083201791
Выборки равны


>Registred in 2020 year<
S: 0.22330639504563676
P: 0.8233608329496916
Выборки равны

t-test in Vladivostok

>Registred in 2021 year<
S: -0.3881225635878612
P: 0.698149755245973
Выборки равны


>Registred in 2020 year<
S: 2.236758419555644
P: 0.025982135601809887
Выборки не равны
Контрольная группа совершила больше по

При углубленной сегментации выяснилось, что только во Владивостоке и лишь для клиентов с регистрацией 2020 года СМС-уведомления приводят к большей конверсии чем при Push-уведомлениях. Следовательно, затраты на СМС-информирование в других сегментах не целесообразны и приведут скорее к убыткам